In [12]:
import pandas as pd
import numpy as np
import datetime
import calendar

In [13]:
#Load all Files (hey must be in input directory in a brother directory of the notebook)
data_load = {
    'air_reserve': pd.read_csv('../input/air_reserve.csv',parse_dates=['visit_datetime','reserve_datetime']), 
    'hpg_reserve': pd.read_csv('../input/hpg_reserve.csv',parse_dates=['visit_datetime','reserve_datetime']), 
    'air_store': pd.read_csv('../input/air_store_info.csv'),
    'hpg_store': pd.read_csv('../input/hpg_store_info.csv'),
    'air_visit': pd.read_csv('../input/air_visit_data.csv',parse_dates=['visit_date']),
    'store_id': pd.read_csv('../input/store_id_relation.csv'),
    'sample_sub': pd.read_csv('../input/sample_submission.csv'),
    'holiday_dates': pd.read_csv('../input/date_info.csv',parse_dates=['calendar_date']).rename(columns={'calendar_date':'visit_date'})
    }

In [14]:
#del Data_sub

Data_sub = data_load['sample_sub'].drop('visitors', axis = 1)

In [15]:
# Split the id to get the store id and the date
Data_sub['air_store_id'] = Data_sub['id'].apply(lambda x: x.split('_')[0]+'_'+x.split('_')[1])

Data_sub['visit_date'] = Data_sub['id'].apply(lambda x: np.datetime64(datetime.date(
    int(x.split('_')[2].split('-')[0]),
    int(x.split('_')[2].split('-')[1]),
    int(x.split('_')[2].split('-')[2])
)))

In [16]:
Data_visit = data_load['air_visit'].copy()

In [17]:
# Adding the mean of visit per day, the max per day, the min per day

D_mean = Data_visit.groupby(['air_store_id']).apply(lambda x: x[(datetime.date(2017,1, 1) < x['visit_date']) & (x['visit_date']< datetime.date(2017,3, 14))]['visitors'].mean()).reset_index(name = 'visit_mean')
D_max = Data_visit.groupby(['air_store_id']).apply(lambda x: x[(datetime.date(2017,1, 1) < x['visit_date']) & (x['visit_date']< datetime.date(2017,3, 14))]['visitors'].max()).reset_index(name = 'visit_max')
D_min = Data_visit.groupby(['air_store_id']).apply(lambda x: x[(datetime.date(2017,1, 1) < x['visit_date']) & (x['visit_date']< datetime.date(2017,3, 14))]['visitors'].min()).reset_index(name = 'visit_min')

Data_sub = Data_sub.merge( 
                      D_mean,
                      how = 'left', 
                      on='air_store_id').merge(D_max,
                                               how = 'left',
                                               on='air_store_id').merge(D_min,
                                                                        how = 'left',
                                                                        on='air_store_id')

In [18]:
# Add the day information

Data_holidays = data_load['holiday_dates'].copy()

# Create a column with the next day date
Data_holidays['next_day_date'] = Data_holidays['visit_date']+datetime.timedelta(1)

# Create the holiday_eve_flg feature
Data_holidays = pd.merge(
                    Data_holidays.set_index("next_day_date",drop=True),
                    data_load['holiday_dates'].drop('day_of_week', axis = 1).set_index("visit_date",drop=True).rename(columns={"holiday_flg": "holiday_eve_flg"}),
                    how = 'left', 
                    left_index = True,
                    right_index = True
                ).reset_index(drop = True).fillna(0)

Data_sub = Data_sub.merge(Data_holidays, how = 'left', on='visit_date')

# Add the store information
Data_sub = Data_sub.merge(data_load['air_store'], how = 'left', on='air_store_id')

# Remove the latitude, longitude features
Data_sub = Data_sub.drop(['latitude', 'longitude'], axis = 1)

# Transform the categorical features
Data_sub_1H = pd.get_dummies(Data_sub, columns=['air_genre_name', 'day_of_week', 'air_area_name' ])

In [19]:
Data_sub_1H

,id,air_store_id,visit_date,visit_mean,visit_max,visit_min,holiday_flg,holiday_eve_flg,air_genre_name_Asian,air_genre_name_Bar/Cocktail,...,air_area_name_Ōsaka-fu Sakai-shi Minamikawaramachi,air_area_name_Ōsaka-fu Suita-shi Izumichō,air_area_name_Ōsaka-fu Ōsaka-shi Fuminosato,air_area_name_Ōsaka-fu Ōsaka-shi Kyōmachibori,air_area_name_Ōsaka-fu Ōsaka-shi Kyūtarōmachi,air_area_name_Ōsaka-fu Ōsaka-shi Nakanochō,air_area_name_Ōsaka-fu Ōsaka-shi Nanbasennichimae,air_area_name_Ōsaka-fu Ōsaka-shi Shinmachi,air_area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,air_area_name_Ōsaka-fu Ōsaka-shi Ōhiraki
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,air_00a91d42b08b08d9_2017-04-28,air_00a91d42b08b08d9,2017-04-28,27.649123,52.0,7.0,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,air_00a91d42b08b08d9_2017-04-29,air_00a91d42b08b08d9,2017-04-29,27.649123,52.0,7.0,1,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,air_00a91d42b08b08d9_2017-04-30,air_00a91d42b08b08d9,2017-04-30,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,air_00a91d42b08b08d9_2017-05-01,air_00a91d42b08b08d9,2017-05-01,27.649123,52.0,7.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,air_00a91d42b08b08d9_2017-05-02,air_00a91d42b08b08d9,2017-05-02,27.649123,52.0,7.0,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
Data_sub_1H.describe(include = 'all')

,id,air_store_id,visit_date,visit_mean,visit_max,visit_min,holiday_flg,holiday_eve_flg,air_genre_name_Asian,air_genre_name_Bar/Cocktail,...,air_area_name_Ōsaka-fu Sakai-shi Minamikawaramachi,air_area_name_Ōsaka-fu Suita-shi Izumichō,air_area_name_Ōsaka-fu Ōsaka-shi Fuminosato,air_area_name_Ōsaka-fu Ōsaka-shi Kyōmachibori,air_area_name_Ōsaka-fu Ōsaka-shi Kyūtarōmachi,air_area_name_Ōsaka-fu Ōsaka-shi Nakanochō,air_area_name_Ōsaka-fu Ōsaka-shi Nanbasennichimae,air_area_name_Ōsaka-fu Ōsaka-shi Shinmachi,air_area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,air_area_name_Ōsaka-fu Ōsaka-shi Ōhiraki
count,32019,32019,32019,31824.000000,31824.000000,31824.000000,32019.000000,32019.000000,32019.000000,32019.000000,...,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000,32019.000000
unique,32019,821,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,air_7ef9a5ea5c8fe39f_2017-05-31,air_d54d6fcb116fbed3,2017-05-26 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,39,821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,2017-04-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2017-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,20.146266,50.143382,3.642157,0.102564,0.102564,0.002436,0.096224,...,0.002436,0.002436,0.002436,0.002436,0.023143,0.002436,0.002436,0.012180,0.030451,0.004872
std,NaN,NaN,NaN,11.471826,41.451163,5.336730,0.303393,0.303393,0.049297,0.294903,...,0.049297,0.049297,0.049297,0.049297,0.150358,0.049297,0.049297,0.109692,0.171827,0.069631
min,NaN,NaN,NaN,1.058824,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,10.828287,29.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### It remains some store_id for which there are no visits registered for the reference period: we will put that at 0 

In [21]:
import pickle

DATA_SUBMISSSION_FILE = "../../data/Data_sub-20171216-2"

Data_sub_1H.to_pickle(DATA_SUBMISSSION_FILE)